In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import RandomForestRegressor

from sklearn.ensemble import ExtraTreesClassifier 

In [ ]:
os.chdir("F:\DATASCIENCE\EDWISOR\Project\Bike RENTAL")

In [ ]:
data = pd.read_csv('day.csv')

In [ ]:
data.head()

In [ ]:
data.dtypes

In [ ]:
data.shape

In [ ]:
data.describe().transpose()

In [ ]:
data = data.drop(['instant'],axis=1)

In [ ]:
data.dtypes

In [ ]:
#histogram 
data.hist(figsize=(15,20))
plt.show()

In [ ]:
data.dtypes

Missing Values

In [ ]:
data.isna().sum()

BOXPLOT FOR OUTLIER ANALYSIS


In [ ]:

data.plot(kind='box',subplots=True,layout=(8,2),sharex=False,sharey=False,figsize=(25,30))
plt.show()


REMOVING OUTLIERS


In [ ]:
cnames = ["temp","atemp","hum","windspeed","casual","registered","cnt"]

#Select and Remove the outliers

for i in cnames:
    q75 , q25 = np.percentile(data.loc[:,i],[75,25])
    iqr = q75  -  q25
    
    min = q25 - (iqr*1.5)
    max = q75 + (iqr*1.5)
    
    print(min)
    print(max)
    
    data = data.drop(data[data.loc[:,i] < min].index)
    data = data.drop(data[data.loc[:,i] > max].index)

print(min)

print(max)

PLOTTING NUMERICAL VARIABLES



In [ ]:
sns.relplot(x='windspeed',y='cnt',data=data)

In [ ]:
sns.relplot(x='windspeed',y='casual',data=data)

In [ ]:
sns.relplot(x='windspeed',y='registered',data=data)

In [ ]:
sns.jointplot(x='temp',y='cnt',kind='hex',data=data)


In [ ]:
sns.relplot(x='temp',y='registered',data=data)

In [ ]:
sns.relplot(x='temp',y='registered',hue='workingday',data=data)

In [ ]:
sns.relplot(x='temp',y='casual',hue='workingday',data=data)

In [ ]:
sns.relplot(x='hum',y='cnt',data=data)

In [ ]:
sns.relplot(x='hum',y='registered',data=data)

In [ ]:
sns.relplot(x='hum',y='casual',data=data)

In [ ]:
sns.relplot(x='windspeed',y='casual',hue='workingday',data=data)

HEATMAP FOR CORRELATION ANALYSIS

In [ ]:
plt.figure(figsize=(15,15))

sns.heatmap(data.corr(),annot=True,cbar = True,cmap="Blues")

Convert few features to categorical for better understanding the data

In [ ]:

to_cat = ['season','yr','weekday','workingday','weathersit','mnth','holiday']
for i in to_cat:
    data[i] = data[i].astype('category')
   

# CATEGORICAL PLOTTING

In [ ]:
sns.catplot(x='yr',y='registered',kind='bar',data=data)

In [ ]:
sns.catplot(x='yr',y='casual',kind='bar',data=data)

In [ ]:
sns.catplot(x='season',y='registered',kind='bar',hue='weathersit',data=data)

In [ ]:
sns.catplot(x='season', y='casual', hue='weathersit',kind='bar',data=data)

In [ ]:
sns.catplot(x='workingday',y='casual',kind='bar',data=data)

In [ ]:
sns.catplot(x='workingday',y='registered',kind='bar',data=data)

In [ ]:
sns.catplot(x='workingday',y='cnt',kind='bar',data=data)

In [ ]:
sns.catplot(x='mnth',y='casual',kind='box',hue='season',aspect=20/8,data=data)

In [ ]:
sns.catplot(x='mnth',y='registered',kind='box',aspect=20/8,hue='season',data=data)

In [ ]:
sns.catplot(x='weekday',y='registered',data=data)

In [ ]:
sns.catplot(x='weekday',y='casual',data=data)

In [ ]:
sns.catplot(x='weekday',y='cnt',data=data)

In [ ]:
data.dtypes

# CREATING ONE HOT ENCODER VARIABLES

In [ ]:
to_cat = ['season','weathersit','yr','weekday']

In [ ]:
for i in to_cat:
    data = pd.get_dummies(data,columns=[i])




In [ ]:
data.head()

In [ ]:
data.shape

# FEATURE SELECTION FOR PREDICTOR AND TARGET SET

In [ ]:
x = data.drop(['dteday','cnt','temp','registered','casual','mnth'],axis=1)

In [ ]:
y = data[['cnt']]


In [ ]:
y.head()

# MODEL BUILDING

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=124)

In [ ]:
y_train.shape

# LINEAR REGRESSION



In [ ]:

lm = LinearRegression().fit(x_train,y_train)


In [ ]:
#PREDICTIONS ON TEST DATA
p = lm.predict(x_test)

In [ ]:
r2_score(y_test,p)

In [ ]:
np.sqrt(mean_squared_error(y_test,p))


In [ ]:
plt.scatter(y_test,p)
plt.xlabel("TrueValues")
plt.ylabel("Predictedvalues")
plt.show()

# RIDGE REGRESSION

In [ ]:
ridge = RidgeCV(alphas=[0.01],cv=5).fit(x_train,y_train)


In [ ]:
ridge.alpha_

In [ ]:
#PREDICTIONS ON TEST DATA
pred  = ridge.predict(x_test)


In [ ]:
np.sqrt(mean_squared_error(y_test,pred))


# RANDOM FOREST


In [ ]:
rf = RandomForestRegressor(n_estimators = 200,bootstrap=True,max_depth=35,oob_score=True, random_state = 42)
# Train the model on training data
rf.fit(x_train, y_train)

In [ ]:
#PREDICTIONS ON TEST DATA
pre = rf.predict(x_test)

In [ ]:
rf.score(x_train,y_train)

In [ ]:
np.sqrt(mean_squared_error(y_test,pre))

In [ ]:
feat_importances = pd.Series(rf.feature_importances_*100, index=x_train.columns)


In [ ]:
feat_importances.nlargest(10).plot(kind='barh')

In [ ]:
plt.scatter(y_test,pre)
plt.xlabel("TrueValues")
plt.ylabel("Predictedvalues")
plt.show()